In [3]:
import pandas as pd

df = pd.read_csv('spotify_sim.csv')

#drop all columns except for the first two

df = df.drop(df.columns[2:], axis=1)

C:\Users\arush\AppData\Local\Temp\ipykernel_7640\2455987336.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('spotify_sim.csv')


In [4]:
# print all the names of the columns

print(df.columns)

#edit the column names to be more descriptive

df.columns = ['artist', 'song']

print(df.columns)

#remove all duplicate artists
if df.duplicated('artist').any():
    df = df.drop_duplicates('artist')


Index([' artistname', ' trackname'], dtype='object')
Index(['artist', 'song'], dtype='object')
                                   artist  \
0                          Elvis Costello   
1        Elvis Costello & The Attractions   
2                            Tiffany Page   
5                                  Lissie   
6                          Paul McCartney   
...                                   ...   
1048542   The Joe Gibbs Family of Artists   
1048543                    The RAS Family   
1048550            Furious George Hartwig   
1048558                     Scott McCurry   
1048564                Family and Friends   

                                                      song  
0                     (The Angels Wanna Wear My) Red Shoes  
1        (What's So Funny 'Bout) Peace, Love And Unders...  
2                                         7 Years Too Late  
5                                              All Be Okay  
6                                          Band On The Run 

In [5]:
df

,artist,song
0,Elvis Costello,(The Angels Wanna Wear My) Red Shoes
1,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders..."
2,Tiffany Page,7 Years Too Late
5,Lissie,All Be Okay
6,Paul McCartney,Band On The Run
...,...,...
1048542,The Joe Gibbs Family of Artists,We Three Kings
1048543,The RAS Family,We Wish You A Merry Christmas
1048550,Furious George Hartwig,"You're a Mean One, Mr. Grinch"
1048558,Scott McCurry,Back To Basics (Honey Do)


In [6]:
# resave the dataframe to a new csv file

df.to_csv('GOOD_spotify_sim.csv', index=False)

In [7]:
import pandas as pd
import requests

def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    print("token acquired")
    return auth_response_data['access_token']

def search_artist(access_token, artist_name):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'q': artist_name,
        'type': 'artist',
        'limit': 1
    }
    search_response = requests.get(search_url, headers=headers, params=params)
    search_data = search_response.json()
    artists_items = search_data['artists']['items']
    print(f"searching for {artist_name}")
    if not artists_items:
        return None, None
    artist_id = artists_items[0]['id']
    artist_genre = artists_items[0]['genres']
    return artist_id, artist_genre

def update_dataframe(data_frame, access_token):
    artist_ids = []
    genres = []
    for artist_name in data_frame['artist']:
        artist_id, genre = search_artist(access_token, artist_name)
        artist_ids.append(artist_id if artist_id else "Not Found")
        genres.append(genre if genre else ["Not Found"])
        print(f"artist: {artist_name}, id: {artist_id}, genre: {genre}")
        print("updating dataframe")
    data_frame['Artist ID'] = artist_ids
    data_frame['Genre'] = genres

# Use your own Client ID and Client Secret
client_id = 'b9f5c171f34f451ea88044ffe1030406'
client_secret = '1e3bb040d2934b9fb88a6e94b2dcbe72'

# Get access token
access_token = get_spotify_token(client_id, client_secret)

# Load the CSV file
file_path = 'path_to_your_csv_file.csv'
spotify_data = pd.read_csv("GOOD_spotify_sim.csv")

# Ensure correct column names
if 'artist' not in spotify_data.columns:
    print("Expected 'Artist Name' column not found. Available columns:", spotify_data.columns)
else:
    update_dataframe(spotify_data, access_token)

# Save the updated DataFrame
spotify_data.to_csv('updated_spotify_data.csv', index=False)

token acquired
searching for Elvis Costello
artist: Elvis Costello, id: 2BGRfQgtzikz1pzAD0kaEn, genre: ['new wave', 'new wave pop', 'permanent wave', 'power pop', 'singer-songwriter']
updating dataframe
searching for Elvis Costello & The Attractions
artist: Elvis Costello & The Attractions, id: 4qmHkMxr6pTWh5Zo74odpH, genre: ['art rock', 'heartland rock', 'new romantic', 'new wave', 'new wave pop', 'power pop']
updating dataframe
searching for Tiffany Page
artist: Tiffany Page, id: 1Q6EgKHZ8mt7Hm5bKa6DbG, genre: []
updating dataframe
searching for Lissie
artist: Lissie, id: 3j4FHbC5zwmYGJ7r0ZgaMt, genre: ['folk-pop']
updating dataframe
searching for Paul McCartney
artist: Paul McCartney, id: 4STHEaNw4mPZ2tzheohgXB, genre: ['classic rock', 'mellow gold', 'rock', 'soft rock']
updating dataframe
searching for Joe Echo
artist: Joe Echo, id: 2bekbCTQa7ve7sz4z4RvPK, genre: []
updating dataframe
searching for The Breakers
artist: The Breakers, id: 3O5xJKrR1BMeB0Z9YM77wp, genre: []
updating da

SSLError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/search?q=Movie+Best+Themes&type=artist&limit=1 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))

Try #1

In [9]:
import pandas as pd
import requests
from requests.exceptions import SSLError

def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    print("token acquired")
    return auth_response_data['access_token']

def search_artist(access_token, artist_name):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'q': artist_name,
        'type': 'artist',
        'limit': 1
    }
    try:
        search_response = requests.get(search_url, headers=headers, params=params)
        search_data = search_response.json()
        artists_items = search_data['artists']['items']
        if not artists_items:
            return None, None
        artist_id = artists_items[0]['id']
        artist_genre = artists_items[0]['genres']
        return artist_id, artist_genre
    except SSLError as ssl_error:
        print(f"SSL Error occurred: {ssl_error}")
        return "SSL Error", "SSL Error"

def update_dataframe(data_frame, access_token):
    artist_ids = []
    genres = []
    try:
        for artist_name in data_frame['artist']:
            artist_id, genre = search_artist(access_token, artist_name)
            artist_ids.append(artist_id if artist_id else "Not Found")
            genres.append(genre if genre else ["Not Found"])
    except Exception as e:
        print(f"Error: {e}")
    finally:
        data_frame['Artist ID'] = artist_ids
        data_frame['Genre'] = genres
        print(f"artist: {artist_name}, id: {artist_id}, genre: {genre}")
        print("updating dataframe")
        data_frame.to_csv('partial_save_1.csv', index=False)
        print("Data partially saved due to an error.")

# Use your own Client ID and Client Secret
client_id = '9e76468d373f4ff1876c927af270ee4c'
client_secret = '1d266b1416784e12bd6df3ab32916dbe'

# Get access token
access_token = get_spotify_token(client_id, client_secret)

# Load the CSV file
file_path = 'Good_spotify_sim.csv'
spotify_data = pd.read_csv(file_path)

# Ensure correct column names
if 'artist' not in spotify_data.columns:
    print("Expected 'Artist Name' column not found. Available columns:", spotify_data.columns)
else:
    update_dataframe(spotify_data, access_token)

# Save the final DataFrame
spotify_data.to_csv('updated_spotify_data.csv', index=False)
print("Data successfully updated and saved.")


token acquired
SSL Error occurred: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/search?q=Daniel+Powter&type=artist&limit=1 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))


ValueError: Length of values (2099) does not match length of index (68569)

In [11]:
import pandas as pd
import requests
from requests.exceptions import SSLError

def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    print("token acquired")
    return auth_response_data['access_token']

def search_artist(access_token, artist_name):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'q': artist_name,
        'type': 'artist',
        'limit': 1
    }
    try:
        search_response = requests.get(search_url, headers=headers, params=params)
        print(f"searching for {artist_name}")
        search_data = search_response.json()
        print("got search data")
        artists_items = search_data['artists']['items']
        print("got artists items")
        if not artists_items:
            return None, None
        artist_id = artists_items[0]['id']
        print("got artist id")
        artist_genre = artists_items[0]['genres']
        print("got artist genre")
        return artist_id, artist_genre
    except SSLError as ssl_error:
        print(f"SSL Error occurred: {ssl_error}")
        return "SSL Error", "SSL Error"

def update_dataframe(data_frame, access_token):
    artist_ids = []
    print("created artist_ids")
    genres = []
    print("created genres")
    for artist_name in data_frame['artist']:
        print(f"updating {artist_name}")
        artist_id, genre = search_artist(access_token, artist_name)
        print(f"artist: {artist_name}, id: {artist_id}, genre: {genre}")
        if artist_id == "SSL Error":
            print("Skipping due to SSL error")
            continue
        artist_ids.append(artist_id if artist_id else "Not Found")
        print("appended artist id")
        genres.append(genre if genre else ["Not Found"])
        print("appended genre")
    data_frame['Artist ID'] = artist_ids
    print("added artist id to dataframe")
    data_frame['Genre'] = genres
    print("added genre to dataframe")

# Use your own Client ID and Client Secret
client_id = 'e848c06844924aaaa446fd8eb330bc18'
client_secret = 'd39ecb0846de4813b44989a3d7ed9e23'

# Get access token
access_token = get_spotify_token(client_id, client_secret)

# Load the CSV file
file_path = 'Good_spotify_sim.csv'
spotify_data = pd.read_csv(file_path)

# Ensure correct column names
if 'artist' not in spotify_data.columns:
    print("Expected 'artist' column not found. Available columns:", spotify_data.columns)
else:
    update_dataframe(spotify_data, access_token)

# Save the final DataFrame
spotify_data.to_csv('updated_spotify_data.csv', index=False)
print("Data successfully updated and saved.")


token acquired
created artist_ids
created genres
updating Elvis Costello
searching for Elvis Costello
got search data
got artists items
got artist id
got artist genre
artist: Elvis Costello, id: 2BGRfQgtzikz1pzAD0kaEn, genre: ['new wave', 'new wave pop', 'permanent wave', 'power pop', 'singer-songwriter']
appended artist id
appended genre
updating Elvis Costello & The Attractions
searching for Elvis Costello & The Attractions
got search data
got artists items
got artist id
got artist genre
artist: Elvis Costello & The Attractions, id: 4qmHkMxr6pTWh5Zo74odpH, genre: ['art rock', 'heartland rock', 'new romantic', 'new wave', 'new wave pop', 'power pop']
appended artist id
appended genre
updating Tiffany Page
searching for Tiffany Page
got search data
got artists items
got artist id
got artist genre
artist: Tiffany Page, id: 1Q6EgKHZ8mt7Hm5bKa6DbG, genre: []
appended artist id
appended genre
updating Lissie
searching for Lissie
got search data
got artists items
got artist id
got artist ge

KeyboardInterrupt: 

In [12]:
import pandas as pd
import requests
from requests.exceptions import SSLError, RequestException
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    print("token acquired")
    return auth_response_data['access_token']

def search_artist(access_token, artist_name):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'q': artist_name,
        'type': 'artist',
        'limit': 1
    }
    try:
        search_response = requests.get(search_url, headers=headers, params=params)
        print(f"searching for {artist_name}")
        search_response.raise_for_status()  # Raises stored HTTPError, if one occurred
        print("got search response")
        search_data = search_response.json()
        print("got search data")
        artists_items = search_data['artists']['items']
        print("got artists items")
        if not artists_items:
            print(f"{artist_name} not found")
            return artist_name, None, None
        artist_id = artists_items[0]['id']
        print("got artist id")
        artist_genre = artists_items[0]['genres']
        print("got artist genre")
        return artist_name, artist_id, artist_genre
    except (SSLError, RequestException) as error:
        print(f"Error fetching {artist_name}: {error}")
        return artist_name, "Error", "Error"

def fetch_artists_data(access_token, artists_names):
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(search_artist, access_token, name) for name in artists_names]
        for future in as_completed(futures):
            results.append(future.result())
    return results

# Use your own Client ID and Client Secret
client_id = '9e76468d373f4ff1876c927af270ee4c'
client_secret = '1d266b1416784e12bd6df3ab32916dbe'

# Get access token
access_token = get_spotify_token(client_id, client_secret)

# Load the CSV file
file_path = 'Good_spotify_sim.csv'
spotify_data = pd.read_csv(file_path)

# Ensure correct column names
if 'artist' not in spotify_data.columns:
    print("Expected 'artist' column not found. Available columns:", spotify_data.columns)
else:
    artist_names = spotify_data['artist'].tolist()
    artist_data = fetch_artists_data(access_token, artist_names)
    # Update DataFrame with fetched data
    for artist_name, artist_id, genre in artist_data:
        spotify_data.loc[spotify_data['artist'] == artist_name, 'Artist ID'] = artist_id
        spotify_data.loc[spotify_data['artist'] == artist_name, 'Genre'] = ', '.join(genre) if genre else "Not Found"

# Save the final DataFrame
spotify_data.to_csv('updated_spotify_data.csv', index=False)
print("Data successfully updated and saved.")


token acquired
searching for Elvis Costello & The Attractions
got search response
got search data
got artists items
got artist id
got artist genre
searching for The Coronas
got search response
got search data
got artists items
got artist id
got artist genre
searching for Joe Echo
got search response
got search data
got artists items
got artist id
got artist genre
searching for Paul McCartney
got search response
got search data
got artists items
got artist id
got artist genre
searching for Joshua Radin
got search response
got search data
got artists items
got artist id
got artist genre
searching for Crowded House
got search response
got search data
got artists items
got artist id
got artist genre
searching for The Breakers
got search response
got search data
got artists items
got artist id
got artist genre
searching for Tiffany Page
got search response
got search data
got artists items
got artist id
got artist genre
searching for Lissie
got search response
got search data
got artists it